In [ ]:
#Dependancies
%matplotlib inline
import matplotlib
import requests
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

from config import college_scorecard_api_key


In [ ]:
# Call in 'College Scorecard (i.e. csc)' API ; only schools that offer a bachelor's or graduate degree
csc_url = "https://api.data.gov/ed/collegescorecard/v1/schools.json?school.degrees_awarded.highest=3,4&"

csc_2017_query_url = f'{csc_url}api_key={college_scorecard_api_key}&_fields=id,school.name,school.city,school.state_fips,school.zip,location.lat,location.lon,2017'
csc_2017_response = requests.get(csc_2017_query_url).json()

# Identify JSON page number
csc_json_data = []

# Loop through each JSON page and extract JSON data into a list
for page in range(0, 29):
    csc_2017_response = requests.get(csc_2017_query_url, params={'page': page, 'per_page': 100}).json()
    csc_json_data.append(csc_2017_response['results'])

    csc_data_list = []
    for csc_data in csc_json_data:
        for csc_objects in csc_data:
            csc_data_list.append(csc_objects)

# Convert JSON list into a Dataframe
csc_data_frame = pd.DataFrame(csc_data_list)


In [ ]:
# Pull columns needed
csc_data_table_pull = csc_data_frame[['school.name',
                                     'school.city',
                                     'school.state_fips',
                                     'location.lat',
                                     'location.lon',
                                     '2017.student.size',
                                     '2017.cost.attendance.academic_year',
                                     '2017.cost.tuition.in_state',
                                     '2017.cost.tuition.out_of_state',
                                     '2017.student.retention_rate.four_year.full_time',
                                     '2017.aid.federal_loan_rate',
                                     '2017.aid.loan_principal',
                                     '2017.aid.median_debt.completers.overall',
                                     '2017.aid.median_debt.income.0_30000',
                                     '2017.aid.median_debt.income.30001_75000',
                                     '2017.aid.median_debt.income.greater_than_75000',
                                     '2017.aid.median_debt.female_students',
                                     '2017.aid.median_debt.male_students',
                                     '2017.completion.rate_suppressed.overall']].dropna().reset_index(drop=True)

# Clean column headers
csc_data_table_header_clean = csc_data_table_pull.rename(columns = {'school.name': 'School Name',
                                                              'school.city': 'City',
                                                              'school.state_fips': 'State Code',
                                                              'location.lat': 'Latitude',
                                                              'location.lon': 'Longitude',
                                                              '2017.student.size': 'Student Size',
                                                              '2017.cost.attendance.academic_year': 'Cost of Attendance',
                                                              '2017.cost.tuition.in_state': 'In-State Tuition',
                                                              '2017.cost.tuition.out_of_state': 'Out-of-State Tuition',
                                                              '2017.student.retention_rate.four_year.full_time': 'Retention Rate: Fulltime Student',
                                                              '2017.aid.federal_loan_rate': '% of Undergrad Students Recieving Fed Student Loan',
                                                              '2017.aid.loan_principal': 'Median Loan Principal',
                                                              '2017.aid.median_debt.completers.overall': 'Median Debt',
                                                              '2017.aid.median_debt.income.0_30000': 'Median Debt with income between 0-30k',
                                                              '2017.aid.median_debt.income.30001_75000': 'Median Debt with income between 30,001-75k',
                                                              '2017.aid.median_debt.income.greater_than_75000': 'Median Debt with income greater than 75k',
                                                              '2017.aid.median_debt.female_students': 'Median Debt for Female Students',
                                                              '2017.aid.median_debt.male_students': 'Median Debt for Male Students',
                                                              '2017.completion.rate_suppressed.overall': 'Completion Rate'})


In [ ]:
# Pull in and merge State names.

fips_path = "Resources/State FIPS Codes.csv"
fips_states_df = pd.read_csv(fips_path)

csc_data_final = pd.merge(csc_data_table_header_clean, fips_states_df, on='State Code')

### Henock's Code - Start

In [ ]:
csc_data_50_states = csc_data_final.loc[csc_data_final["State Name"] != 'District of Columbia', :]
#csc_data_50_states

In [ ]:
# Group by State - Averages
csc_grouped_data = csc_data_50_states.groupby(['State Name']).mean()

# Avg. Cost of Attendance by State
csc_sorted_data_ca = csc_grouped_data.sort_values('Cost of Attendance', ascending = False)
avg_ca_by_state = csc_sorted_data_ca['Cost of Attendance'].reset_index()
top_ten_avg_ca_by_state = avg_ca_by_state.head(10)

fig, ax = plt.subplots(figsize = (7, 5))

state_names_x = top_ten_avg_ca_by_state['State Name'].tolist()
ca_debt_y = top_ten_avg_ca_by_state['Cost of Attendance'].tolist()
y_pos = np.arange(len(state_names_x))

ax.barh(y_pos, ca_debt_y, align='center')
ax.set_yticks(y_pos)
ax.set_yticklabels(state_names_x)
ax.invert_yaxis() 
ax.set_xlabel('Cost of Attendance')
ax.set_title('Avg. Cost of Attendance by State')
plt.xlim(20000, max(ca_debt_y)+1000)
plt.ylim(10, -1)


# Avg. Median Debt by State
csc_sorted_data_md = csc_grouped_data.sort_values('Median Debt', ascending = False)
avg_md_state = csc_sorted_data_md['Median Debt'].reset_index()
top_ten_avg_md_by_state = avg_md_state.head(10)

fig, ax = plt.subplots(figsize = (7, 5))

state_names_x = top_ten_avg_md_by_state['State Name'].tolist()
md_debt_y = top_ten_avg_md_by_state['Median Debt'].tolist()
y_pos = np.arange(len(state_names_x))

ax.barh(y_pos, md_debt_y, align='center')
ax.set_yticks(y_pos)
ax.set_yticklabels(state_names_x)
ax.invert_yaxis() 
ax.set_xlabel('Median Debt')
ax.set_title('Avg. Median Debt by State')
plt.xlim(20000, max(md_debt_y)+1000)
plt.ylim(10, -1)

In [ ]:
# Poulation data for t-Test

csc_ca_population = csc_data_final['Cost of Attendance']
csc_md_population = csc_data_final['Median Debt']

# Scatter Plot of Data
plt.subplot(2, 1, 1)
plt.scatter(range(len(csc_ca_population)), csc_ca_population, label="Cost of Attendance")
plt.scatter(range(len(csc_md_population)), csc_md_population, label="Median Debt")
plt.legend()

# Histogram Plot of Data
plt.subplot(2, 1, 2)
plt.hist(csc_ca_population, 10, density=True, alpha=0.7, label="Cost of Attendance")
plt.hist(csc_md_population, 10, density=True, alpha=0.7, label="Median Debt")
plt.axvline(csc_ca_population.mean(), color='k', linestyle='dashed', linewidth=1)
plt.axvline(csc_md_population.mean(), color='k', linestyle='dashed', linewidth=1)
plt.legend() 


In [ ]:
stats.ttest_ind(csc_ca_population, csc_md_population, equal_var=False)

### Henock's Code - End

----------------------------------------------

### JJ's Code - Start

### JJ's Code - End

----------------------------------------------

### Ana's Code - Start

### Ana's Code - End

----------------------------------------------

### Vikash's Code - Start

### Vikash's Code - End